## Path Planning Evaluation: Outward Spiral vs Bayesian

In [1]:
import h3
import folium
import numpy as np
import os
import random

from clusterfinder.clusterfinder import (
    DIANAClusterFinder
)
from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.viz import (
    add_hex_to_map,
)
from experiment.test_framework import TestFramework

# Run and validate
RES=15
NUM_HOTSPOT = 50
NUM_CASUALTY = 50
STEPS = 500
CENTRE = (1.3392911509416838, 103.95958286190708)
MAP = folium.Map(location=CENTRE, zoom_start=16, tiles='cartodb positron', max_zoom=24)

### 800 steps

In [ ]:
STEPS = 200

# print("\nSpiral")
# test_object = TestFramework(name="DIANA", res=RES)
# test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
# test_object.register_cluster_finder(DIANAClusterFinder, threshold=0.1)
# test_object.register_path_finder(OutwardSpiralPathFinder)
# test_object.run(STEPS, only_path=True, print_output=False)

print("\nBayes")
test_object = TestFramework(name="DIANA", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DIANAClusterFinder, threshold=0.1)
test_object.register_path_finder(BayesianHexSearch)
test_object.run(STEPS, only_path=True, update_map=True, f=print_output=False)

In [ ]:
def visualize_hotspots_and_casualties(map: folium.Map, hotspots, casualty_locations):
    """
    hotspots: list of tuple of (lat, lng)
    casualty_locations: set of hex_idx
    """
    casualty_locations = [h3.h3_to_geo(hex_idx) for hex_idx in casualty_locations]
    
    for hs in hotspots:
        folium.Marker(location=hs, icon=folium.Icon(color='red', icon='fire')).add_to(map)
    for casualty in casualty_locations:
        folium.Marker(location=casualty, icon=folium.Icon(color='orange', icon='exclamation-sign')).add_to(map)
    return map

def visualize_path(map: folium.Map, search_outputs, max_outputs=3):
    """
    Visualize up to max_outputs paths on a map.
    
    :param map_object: folium.Map object to which the paths will be added
    :param search_outputs: list of dictionaries with 'hex_idx' and 'step_count'
    :param max_outputs: maximum number of paths to visualize
    :return: folium.Map object with paths added
    """
    for idx, output in enumerate(search_outputs.values()):
        if idx >= max_outputs:
            break
        add_hex_to_map(output, map)
    return map


map = test_object.main_map
hotspots, casualty_locations = test_object.hotspots, test_object.casualty_locations
all_search_outputs = test_object.all_search_outputs

map = visualize_hotspots_and_casualties(map, hotspots, casualty_locations)
map = visualize_path(map, all_search_outputs)
display(map)